In [2]:
import sounddevice as sd
import scipy.io.wavfile as wav

# Define the duration of the recording in seconds
duration = 2

# Set the sampling frequency and number of channels
fs = 44100
channels = 1

# Start recording
print('Recording started...')
recording = sd.rec(int(duration * fs), samplerate=fs, channels=channels)

# Wait for the recording to finish
sd.wait()

# Save the recording to a wave file
filename = 'recording.wav'
wav.write(filename, fs, recording)

# Play the audio back out
#print('Playing back...')
#playback = wav.read(filename)
#sd.play(playback[1], playback[0])
#sd.wait()
#print('Playback finished.')

block_len_ms = 32 
block_shift_ms = 8
fs_target = 44100

Recording started...


In [3]:
import numpy as np
import sounddevice as sd
import tensorflow.lite as tflite

# set some parameters
block_len_ms = 32 
block_shift_ms = 8
fs_target = 16000

# create the interpreters
interpreter_1 = tflite.Interpreter(model_path='./pretrained_model/model_1.tflite')
interpreter_1.allocate_tensors()
interpreter_2 = tflite.Interpreter(model_path='./pretrained_model/model_2.tflite')
interpreter_2.allocate_tensors()

# Get input and output tensors.
input_details_1 = interpreter_1.get_input_details()
output_details_1 = interpreter_1.get_output_details()
input_details_2 = interpreter_2.get_input_details()
output_details_2 = interpreter_2.get_output_details()

# create states for the lstms
states_1 = np.zeros(input_details_1[1]['shape']).astype('float32')
states_2 = np.zeros(input_details_2[1]['shape']).astype('float32')

# calculate shift and length
block_shift = int(np.round(fs_target * (block_shift_ms / 1000)))
block_len = int(np.round(fs_target * (block_len_ms / 1000)))

# create buffer
in_buffer = np.zeros((block_len)).astype('float32')
out_buffer = np.zeros((block_len)).astype('float32')


# calculate fft of input block
in_block_fft = np.fft.rfft(in_buffer)
in_mag = np.abs(in_block_fft)
in_phase = np.angle(in_block_fft)
print('calc fft')

# reshape magnitude to input dimensions
in_mag = np.reshape(in_mag, (1,1,-1)).astype('float32')
print('reshape')
print(in_mag.shape)
    
interpreter_1.set_tensor(input_details_1[1]['index'], states_1)
interpreter_1.set_tensor(input_details_1[0]['index'], in_mag)
print('set tensors')

2023-04-23 18:13:43.434787: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-23 18:13:43.436714: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-23 18:13:43.465875: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-23 18:13:43.466255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 18:13:44.044331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

calc fft
reshape
(1, 1, 257)
set tensors


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
import numpy as np
import sounddevice as sd
import soundfile as sf
import librosa

# set some parameters
block_len_ms = 32 
block_shift_ms = 8
fs_input = 44100
fs_target = 16000

# create the interpreters
interpreter_1 = tflite.Interpreter(model_path='./pretrained_model/model_1.tflite')
interpreter_1.allocate_tensors()
interpreter_2 = tflite.Interpreter(model_path='./pretrained_model/model_2.tflite')
interpreter_2.allocate_tensors()

# Get input and output tensors.
input_details_1 = interpreter_1.get_input_details()
output_details_1 = interpreter_1.get_output_details()
input_details_2 = interpreter_2.get_input_details()
output_details_2 = interpreter_2.get_output_details()

# create states for the lstms
states_1 = np.zeros(input_details_1[1]['shape']).astype('float32')
states_2 = np.zeros(input_details_2[1]['shape']).astype('float32')

# calculate shift and length
block_shift = int(np.round(fs_target * (block_shift_ms / 1000)))
block_len = int(np.round(fs_target * (block_len_ms / 1000)))

# create buffer
in_buffer = np.zeros((block_len)).astype('float32')
out_buffer = np.zeros((block_len)).astype('float32')

# resample audio to the target sampling rate
def resample(audio, sr, target_sr):
    resampled = librosa.resample(audio, sr, target_sr)
    return resampled, target_sr

def callback(indata, outdata, frames, time, status):
    print('starting callback')
    # buffer and states to global
    global in_buffer, out_buffer, states_1, states_2
    if status:
        print(status)
    print('passing states')
    
    # resample audio to the target sampling rate
    indata, fs_input = resample(indata, fs_input, fs_target)
        
    # write to buffer
    in_buffer[:-block_shift] = in_buffer[block_shift:]
    in_buffer[-block_shift:] = np.squeeze(indata)
    print('writing to buffer')

    # save a copy of the input audio
#    sf.write('input_audio.wav', indata, fs_target, subtype='PCM_24')
#    print('saving input wav')
    
    # calculate fft of input block
    in_block_fft = np.fft.rfft(in_buffer)
    in_mag = np.abs(in_block_fft)
    in_phase = np.angle(in_block_fft)
    print('calc fft')

    # reshape magnitude to input dimensions
    in_mag = np.reshape(in_mag, (1,1,-1)).astype('float32')
    print('reshape')
    print(in_mag.shape)

    # set tensors to the first model
    interpreter_1.set_tensor(input_details_1[1]['index'], states_1)
    interpreter_1.set_tensor(input_details_1[0]['index'], in_mag)
    print('set tensors')

    # run calculation 
    interpreter_1.invoke()
    print('pass invoke')

    # get the output of the first block
    out_mask = interpreter_1.get_tensor(output_details_1[0]['index']) 
    states_1 = interpreter_1.get_tensor(output_details_1[1]['index'])   
    print('output first block')
    
    # calculate the ifft
    estimated_complex = in_mag * out_mask * np.exp(1j * in_phase)
    estimated_complex = np.concatenate([estimated_complex, np.flip(np.conj(estimated_complex[1:-1]), axis=0)], axis=0)
    estimated_block = np.fft.irfft(estimated_complex)
    print('calc ifft')

    # downsample the estimated block
    downsampled_block = estimated_block[::int(fs_target/16000)]

    # reshape the time domain block
    downsampled_block = np.reshape(downsampled_block, (1,1,-1)).astype('float32')
    print('reshape td block')

    # set tensors to the second block
    interpreter_2.set_tensor(input_details_2[1]['index'], states_2)
    interpreter_2.set_tensor(input_details_2[0]['index'], downsampled_block)
    print('tensors of second block')

    # run calculation
    interpreter_2.invoke()  
    print('int 2 invoke')

    # get output tensors
    out_block = interpreter_2.get_tensor(output_details_2[0]['index']) 
    states_2 = interpreter_2.get_tensor(output_details_2[1]['index'])

    # upsample the output block
    upsampled_block = np.zeros((block_len))
    upsampled_block[::int(fs_target/16000)] = np.squeeze(out_block)

    # write to buffer
    out_buffer[:-block_shift] = out_buffer[block_shift:]
    out_buffer[-block_shift:] = upsampled_block
    print('writting to outbuffer')

    # output to soundcard
    outdata[:] = np.expand_dims(out_buffer[:block_shift], axis=-1)

#choose the default input and output devices
input_device = 23
output_device = 23

#print the chosen devices
print(f"Using input device: {input_device}")
print(f"Using output device: {output_device}")

#open a sound stream with the chosen devices and the callback function
with sd.Stream(device=(input_device, output_device),
    samplerate=fs_target, blocksize=block_shift,
    dtype=np.float32, channels=1, callback=callback):
    print('#' * 80)
    print('Press Ctrl+C to stop')
    print('#' * 80)
    
while True:
    try:
        sd.sleep(100)
    except KeyboardInterrupt:
        print('\nClosing stream')
        break
    except Exception as e:
        print(type(e).name + ': ' + str(e))

Using input device: 23
Using output device: 23
################################################################################
Press Ctrl+C to stop
################################################################################

Closing stream


In [5]:
interpreter_1 = tflite.Interpreter(model_path='./pretrained_model/model_1.tflite')
interpreter_1.allocate_tensors()

input_details = interpreter_1.get_input_details()
output_details = interpreter_1.get_output_details()

print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

interpreter_1.invoke()
print("pass invoke")

Input shape: [  1   1 257]
Output shape: [  1   1 257]
pass invoke


In [6]:
import numpy as np
import sounddevice as sd
import scipy.io.wavfile as wav
import time
import tensorflow.lite as tflite

# Define the window size and shift
fs = 16000  # Set the sampling frequency and number of channels
channels = 1
window_size = int(32 * fs / 1000)  # 32ms window size
shift_size = int(8 * fs / 1000)  # 8ms shift

# Create the interpreters
interpreter_1 = tflite.Interpreter(model_path='./pretrained_model/model_1.tflite')
interpreter_1.allocate_tensors()
interpreter_2 = tflite.Interpreter(model_path='./pretrained_model/model_2.tflite')
interpreter_2.allocate_tensors()

# Start processing the audio in real-time
print('Processing audio in real-time...')
stream = sd.Stream(samplerate=fs, blocksize=shift_size, channels=channels)

# Start the stream
stream.start()

# Initialize the audio buffer and window
audio_buffer = []
window = np.array([])

duration = 5  # Set the recording duration to 5 seconds
start_time = time.time()

try:
    while True:
        # Read a block of audio samples
        block, overflow = stream.read(shift_size)
        print("block shape:", block.shape)

        # Extract the current window and perform inference on model 1
        window = np.concatenate((window, block[:, 0]), axis=0)[-window_size:]
        if len(window) == window_size:
            input_details = interpreter_1.get_input_details()
            output_details = interpreter_1.get_output_details()
            interpreter_1.set_tensor(input_details[0]['index'], np.expand_dims(window.astype(np.float32), axis=0))
            interpreter_1.invoke()
            output_1 = interpreter_1.get_tensor(output_details[0]['index'])

            # Perform inference on model 2 using the output of model 1
            input_details = interpreter_2.get_input_details()
            output_details = interpreter_2.get_output_details()
            interpreter_2.set_tensor(input_details[0]['index'], output_1)
            interpreter_2.invoke()
            output_2 = interpreter_2.get_tensor(output_details[0]['index'])

            # Append the output of model 2 to the audio buffer
            audio_buffer.append(output_2)
            print("output_2 shape:", output_2.shape)

        # Check if the "q" key was pressed or the duration has elapsed
        elapsed_time = time.time() - start_time
        print("elapsed_time:", elapsed_time)
        if np.any(block == b'q') or elapsed_time >= duration:
            if np.any(block == b'q'):
                print('\nRecording stopped with "q" key...')
            else:
                print(f'\nRecording stopped after {duration} seconds...')
            break

except KeyboardInterrupt:
    # Stop the stream when Ctrl-C is pressed
    stream.stop()
    print('\nStream stopped.')

finally:
    # Save the audio buffer to a wave file
    if len(audio_buffer) > 0:
        filename = 'my_recording.wav'
        audio_data = np.concatenate(audio_buffer, axis=0)
        wav.write(filename, fs, audio_data)
        print(f'Audio saved to "{filename}" with shape:', audio_data.shape)
    else:
        print('No audio recorded')  # Add a message if no audio is recorded

    # Stop the stream
    stream.stop()

Processing audio in real-time...
block shape: (128, 1)
elapsed_time: 0.023009777069091797
block shape: (128, 1)
elapsed_time: 0.02329707145690918
block shape: (128, 1)
elapsed_time: 0.06627702713012695
block shape: (128, 1)
No audio recorded


/tmp/ipykernel_12654/3964938109.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if np.any(block == b'q') or elapsed_time >= duration:


ValueError: Cannot set tensor: Dimension mismatch. Got 2 but expected 3 for input 0.